In [29]:
import pandas as pd

df = pd.read_csv("../Daten/swissvotes-dataset.csv", delimiter=";", na_values=".")
# Exclude data from votes with missing values (very first and of the last)
df = df.iloc[1:-12, :]
# Extract some data rows for comparison and verification
volk = df["volk"].iloc[:]
stand = df["stand"].iloc[:]
kt_ja = df["kt-ja"].iloc[:]
kt_nein = df["kt-nein"].iloc[:]
ktjaproz = df["ktjaproz"].iloc[:]
# Do some sanity checks on the data
staende = kt_ja + kt_nein
invalid_results = (staende > 23).sum() + (staende < 22).sum()
print("Number of invalid staende results: {}".format(invalid_results))
print("Number of times the popular count and staende disagree: {}".format((volk != stand).sum()))


Number of invalid staende results: 0
Number of times the popular count and staende disagree: 202


In [30]:
df = df.set_index('anr').transpose()
df_results = df[df.index.str.endswith("-japroz")]
number_cantons = df_results.count()
weights = df_results[3.0].copy()
weights[:] = 1.0
weights.loc[['ar-japroz','ai-japroz','ow-japroz','nw-japroz','bl-japroz','bs-japroz']] = 0.5

In [31]:
cantons_yes = (df_results > 50.0).sum() > number_cantons
cantons_yes = cantons_yes.reset_index(drop=True)
stand = stand.reset_index(drop=True)
disagreements = (cantons_yes == (stand > 0.9)).sum()
print("Number of times the popular count and staende disagree: {}".format(disagreements))

Number of times the popular count and staende disagree: 258


In [32]:
results_b_u = (df_results > 50.0)
results_b_w = (df_results > 50.0).multiply(weights, axis=0)
#kt_ja = kt_ja.reset_index(drop=True)
my_kt_ja_b_u = results_b_u.sum() > number_cantons/2
#my_kt_ja_b_u = my_kt_ja_b_u.reset_index(drop=True)
my_kt_ja_b_w = results_b_w.sum() > (number_cantons-3.0)/2
my_kt_ja_b_w = my_kt_ja_b_w.reset_index(drop=True)

results_p_u = df_results.sum()
results_p_w = (df_results > 50.0).sum()
my_kt_ja_p_u = results_p_u > 50.0*number_cantons
#my_kt_ja_p_u = my_kt_ja_p_u.reset_index(drop=True)
my_kt_ja_p_w = results_p_w > 50.0*(number_cantons-3.0)
#my_kt_ja_p_w = my_kt_ja_p_w.reset_index(drop=True)

same = (results_b_u.sum() == kt_ja)
same.index[same == False]

ValueError: Can only compare identically-labeled Series objects

In [34]:
df.iloc[:, 275]
results_b_w.sum()



anr
2.0       9.5
3.0      12.5
4.0       8.0
5.0       9.0
6.0      10.0
         ... 
624.0     0.0
625.0    21.0
626.0     0.0
627.0    23.0
628.0    22.0
Length: 646, dtype: object